**Welcome to our final project!**

NYT COVID-19 data found here:
https://github.com/nytimes/covid-19-data

Poverty, Unemployment, Population data found here:
https://www.ers.usda.gov/data-products/county-level-data-sets/download-data/

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Importing NYT series data as covid19 and the population estimates by FIPS from phase 1A

In [3]:
covid19 = pd.read_csv('1B_DATA/us-counties.csv')

In [4]:
pop_data = pd.read_csv('1B_DATA/phase1_popdata.csv')

# DEFINE SIRD MODEL

In [5]:
# def run_sird(population, b, g, f, numdays): 
def run_sird(population, b, f, numdays):
    
  # values 
  D0 = 0
  I0 = 1
  R0 = 0
  S0 = population - I0 - D0 

# A grid of time points (in days)
  t = np.linspace(0, numdays, numdays + 1)

  S = S0
  I = I0
  R = R0
  D = D0

  S_list = [S0]
  I_list = [I0]
  R_list = [R0]
  D_list = [D0]

  g = 1/14
  for time in t[1:]:
      dSdt = (-1.0 * b * S * I)
      dIdt = (b * S * I) - (g * I)
      dDdt = g * f * I
      dRdt = ((1-f) * g * I)
      if (S + dSdt < 0):
            dSdt = -1 * S
            dIdt = S - (g * I)
      S = S + dSdt
      I = I + dIdt
      R = R + dRdt
      D = D + dDdt
      S_list.append(S)
      I_list.append(I)
      R_list.append(R)
      D_list.append(D)
  return S_list, I_list, R_list, D_list

# DEFINING the objective function to be minimized

In [6]:
from sklearn.metrics import mean_squared_error

# Note that this cost function is based solely on the death data 

def error_of_sim(deaths, D_test):
    original = pd.DataFrame()
    original['deaths'] = deaths
    original['d_test'] = D_test
    return mean_squared_error(original['deaths'], original['d_test'])



In [7]:
## Takes x = [initial beta, initial g, initial f], args = (temp_col[deaths], population, temP-col['cases'])


#Using this for PSO
def objective_fun2(x, deaths, population):
    #S, I, R, D = run_sird(population, x[0, 0], x[0, 1], x[0, 2], len(cases) - 1)
    S, I, R, D = run_sird(population, x[0,0], x[0,1], len(deaths) - 1)
    result = error_of_sim(deaths, D)
    return result

# INSTALLING AND RUNNING PSO ON EVERY COUNTY WITH 20+ days of COVID19

In [8]:
pip install pyswarms

Note: you may need to restart the kernel to use updated packages.


In [12]:
from IPython.utils import io
from pyswarms.single.global_best import GlobalBestPSO
unique_fips = pd.Series(covid19['fips']).dropna()
count = 0

x_max = np.array([.00002, .05])
x_min = np.zeros(2)
bounds = (x_min, x_max)

options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pop_data["beta"] = np.nan
pop_data["f"] = np.nan
pop_data["cost"] = np.nan
for fips in unique_fips: 
      print(count)
      optimizer = GlobalBestPSO(n_particles=50, dimensions=2, options=options, bounds=bounds)
      population = pop_data[pop_data.FIPS == fips]['POPESTIMATE2019']
      #print(population.iloc[0], " ", fips)
      if(len(population) == 0):
        continue
      else:
        temp_col = covid19[covid19.fips == fips]
        if (len(temp_col) < 20):
            continue
    
        kwargs = {"deaths":temp_col["deaths"], "population":population.iloc[0]} #"cases":temp_col['cases']}

        with io.capture_output() as captured:
            cost, pos = optimizer.optimize(objective_fun2, 1000, **kwargs)
        
        pop_data.loc[pop_data.FIPS == fips, 'beta'] = pos[0]
        pop_data.loc[pop_data.FIPS == fips, 'f'] = pos[1]
        pop_data.loc[pop_data.FIPS == fips, 'cost'] = cost 
        count += 1                                                                                                                                      

2020-05-05 21:17:57,238 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


0


2020-05-05 21:17:59,496 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 663.710104706106, best pos: [2.13342366e-07 7.69964658e-03]
2020-05-05 21:17:59,512 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


1


2020-05-05 21:18:01,705 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 430.2115523194222, best pos: [2.03344189e-07 1.91891527e-02]
2020-05-05 21:18:01,720 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


...

2020-05-05 21:18:20,818 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 813000.7814765415, best pos: [8.05031789e-06 1.20695025e-04]
2020-05-05 21:18:20,834 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


10


2020-05-05 21:18:23,305 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1629.0241798959196, best pos: [1.23699571e-05 1.75905197e-05]
2020-05-05 21:18:23,317 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


11


KeyboardInterrupt: 

## EXPORT

In [ ]:
pop_data.dropna().to_csv("1B_data/data_1b_bf.csv")

# GRAPHS FOR FIRST 100 counties

In [ ]:
for index, row in pop_data.dropna().head(n=100).iterrows():
    temp_col = covid19[covid19.fips == row['FIPS']]
    county, state = temp_col["county"].iloc[0], temp_col["state"].iloc[0]
    title = county + ", " + state
    
    S, I, R, D = run_sird(row['POPESTIMATE2019'], row["beta"], row['f'], len(temp_col) - 1)
    t = np.linspace(0, len(temp_col)-1, len(temp_col))
    

    fig = plt.figure(facecolor='w')
    ax = fig.add_subplot(111, axisbelow=True)
    ax.plot(t, I, 'g', alpha=0.5, lw=2, label='Simulated Infected')
    ax.plot(t, D, 'o', alpha=0.5, lw=2, label='Simulated Deaths')


    ax.plot(t, temp_col['cases'] * (100/82.1), label = 'True Cases')
    ax.plot(t, temp_col['deaths'], 'b', label = 'True Deaths')

    ax.set_xlabel('Time /days')
    ax.set_ylabel('Number')
    ax.set_title(title)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)

    ax.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    plt.show()